In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.sql.streaming.schemaInference", True).getOrCreate()

stream = spark.\
    readStream.\
    format("ws").\
    option("schema", "ticker").\
    load() # we need to pass `option("schema", "ticker")` to get correct channel subscribed

query = stream.select("side", "product_id", "last_size", "best_bid", "best_ask", "time").\
    writeStream.\
    format("console").\
    outputMode("append").\
    option("truncate", "false").\
    start()

query.awaitTermination(10) # Let's wait for 10 seconds.
query.stop() # Let's stop the query
stream.printSchema()
#spark.stop() # And stop the whole session

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/28 15:48:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/28 15:49:02 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-71ca7002-916c-4034-bb87-fce5b17844b1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/11/28 15:49:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----------+---------+--------+--------+----+
|side|product_id|last_size|best_bid|best_ask|time|
+----+----------+---------+--------+--------+----+
+----+----------+---------+--------+--------+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+----+----------+----------+--------+--------+-----------------------+
|side|product_id|last_size |best_bid|best_ask|time                   |
+----+----------+----------+--------+--------+-----------------------+
|buy |ETH-BTC   |0.18862195|0.07507 |0.07508 |2021-11-28 15:48:50.158|
|sell|ETH-USD   |0.10579858|4083.31 |4083.52 |2021-11-28 15:49:03.593|
+----+----------+----------+--------+--------+-----------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+----+----------+----------+--------+--------+-----------------------+
|side|

Uruchamiająć `stream.start()` uruchamiamy w osobnym demonie websocket który streamuje wyniki. Jeżeli wystąpi jakiś błąd po stronie front-endu (np. błąd parsowania kolejnej linijki Pythona) fakt ten nie zostanie zgłoszony do sparka i socket pozostanie otwarty! Należy pamiętać, by zamykać stream za każdym razem używająć metody `stop()` (w powyższym przykładzie `query.stop()`). W przypadku utracenia referencji do zapytania, należy zastopować całą sesję również metodą `stop()` (w powyższym przykładzie `spark.stop()`) 

In [ ]:
# Panic button - press only if you messed up opening new websocket and lost reference to it

query.stop()
spark.stop()

-------------------------------------------
Batch: 158
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.5054119318183 |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.5054119318183 |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.5054119318183 |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4083.989243243242  |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4084.69796

-------------------------------------------
Batch: 159
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.511361702128  |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.511361702128  |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.511361702128  |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4083.9965498652277 |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4084.73133

-------------------------------------------
Batch: 160
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.5420563380285 |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.5420563380285 |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.5420563380285 |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4084.034760638297  |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4084.89584

-------------------------------------------
Batch: 161
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.5549859550565 |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.5549859550565 |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.5549859550565 |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4084.051216931216  |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4084.96298

-------------------------------------------
Batch: 162
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.561598877981  |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.561598877981  |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.561598877981  |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4084.0597097625323 |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508055555555554|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4084.99691

...

-------------------------------------------
Batch: 171
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.705366847826  |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.705366847826  |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.705366847826  |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4084.2372139303475 |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4085.54417

-------------------------------------------
Batch: 172
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.7342105263156 |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.7342105263156 |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.7342105263156 |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4084.270909090908  |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4085.61895

-------------------------------------------
Batch: 173
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.7514650537632 |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.7514650537632 |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.7514650537632 |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4084.290975609755  |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4085.66121

-------------------------------------------
Batch: 174
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.757087248322  |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.757087248322  |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.757087248322  |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4084.297420924573  |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4085.67400

-------------------------------------------
Batch: 175
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:34:00, 2021-11-28 15:44:00}|ETH-USD   |4082.7684337349397 |
|{2021-11-28 15:32:00, 2021-11-28 15:42:00}|ETH-BTC   |0.07507344827586206|
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-USD   |4082.7684337349397 |
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:36:00, 2021-11-28 15:46:00}|ETH-USD   |4082.7684337349397 |
|{2021-11-28 15:30:00, 2021-11-28 15:40:00}|ETH-BTC   |0.075063125        |
|{2021-11-28 15:40:00, 2021-11-28 15:50:00}|ETH-USD   |4084.3104842615    |
|{2021-11-28 15:38:00, 2021-11-28 15:48:00}|ETH-BTC   |0.07508189189189188|
|{2021-11-28 15:42:00, 2021-11-28 15:52:00}|ETH-USD   |4085.69990

# Zadanie 1

**Analiza strumienia danych CoinBase (3p)**. Napisz zapytanie, które wypisuje średnią wartość wybranego parametru (np. `price`) w przesuwnych oknach czasowych względem czasu transakcji (kolumna `time`), grupując po relacji wymiany (z jakiej waluty na jaką walutę - kolumna `product_id`). 

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import window

spark = SparkSession.builder.\
    config("spark.sql.streaming.schemaInference", True).\
    config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True).getOrCreate()  

stream = spark.readStream.\
    format("ws").\
    option("schema", "ticker").\
    load()

query = stream.select("product_id", "price", "time").\
    groupBy(window("time", "2 minutes", "1 minutes"), "product_id").mean("price")\
    .orderBy("window")

query = query.\
    writeStream.\
    format("console").\
    outputMode("complete").\
    option("truncate", "false").\
    start()

query.awaitTermination(200) # Let's wait for 10 seconds.
query.stop() # Let's stop the query



21/11/28 15:50:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1e6a7ac3-d2bb-4c81-977a-618686ec7c2a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/11/28 15:50:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----------+-----------------+
|window                                    |product_id|avg(price)       |
+------------------------------------------+----------+-----------------+
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-BTC   |0.07511          |
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-USD   |4088.296666666667|
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-USD   |4088.296666666667|
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-BTC   |0.07511          |
+------------------------------------------+----------+-----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-BTC   |0.0751             |
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-USD   |4088.463571428572  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-USD   |4088.625806451613  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-BTC   |0.07509666666666666|
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-USD   |4088.7594117647054 |
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-BTC   |0.07509            |
+------------------------------------------+----------+-------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-BTC   |0.0751             |
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-USD   |4088.463571428572  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-USD   |4088.7297619047617 |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-BTC   |0.07509666666666666|
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-USD   |4088.8628571428567 |
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-BTC   |0.07509            |
+------------------------------------------+----------+-------------------+



...

-------------------------------------------
Batch: 66
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-BTC   |0.0751             |
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-USD   |4088.463571428572  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-USD   |4090.093977900553  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-BTC   |0.07509818181818181|
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-USD   |4088.939152119701  |
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-BTC   |0.075113125        |
|{2021-11-28 15:52:00, 2021-11-28 15:54:00}|ETH-BTC   |0.07514214285714285|
|{2021-11-28 15:52:00, 2021-11-28 15:54:00}|ETH-USD   |4087.999074074075  |
|{2021-11-28 15:53:00, 2021-11-28 15:55:00}|ETH-USD   |4087.985629

-------------------------------------------
Batch: 67
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-BTC   |0.0751             |
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-USD   |4088.463571428572  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-USD   |4090.093977900553  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-BTC   |0.07509818181818181|
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-USD   |4088.939152119701  |
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-BTC   |0.075113125        |
|{2021-11-28 15:52:00, 2021-11-28 15:54:00}|ETH-BTC   |0.07514214285714285|
|{2021-11-28 15:52:00, 2021-11-28 15:54:00}|ETH-USD   |4087.999074074075  |
|{2021-11-28 15:53:00, 2021-11-28 15:55:00}|ETH-USD   |4087.989281

-------------------------------------------
Batch: 68
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-BTC   |0.0751             |
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-USD   |4088.463571428572  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-USD   |4090.093977900553  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-BTC   |0.07509818181818181|
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-USD   |4088.939152119701  |
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-BTC   |0.075113125        |
|{2021-11-28 15:52:00, 2021-11-28 15:54:00}|ETH-BTC   |0.07514214285714285|
|{2021-11-28 15:52:00, 2021-11-28 15:54:00}|ETH-USD   |4087.999074074075  |
|{2021-11-28 15:53:00, 2021-11-28 15:55:00}|ETH-USD   |4087.989874

-------------------------------------------
Batch: 69
-------------------------------------------
+------------------------------------------+----------+-------------------+
|window                                    |product_id|avg(price)         |
+------------------------------------------+----------+-------------------+
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-BTC   |0.0751             |
|{2021-11-28 15:49:00, 2021-11-28 15:51:00}|ETH-USD   |4088.463571428572  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-USD   |4090.093977900553  |
|{2021-11-28 15:50:00, 2021-11-28 15:52:00}|ETH-BTC   |0.07509818181818181|
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-USD   |4088.939152119701  |
|{2021-11-28 15:51:00, 2021-11-28 15:53:00}|ETH-BTC   |0.075113125        |
|{2021-11-28 15:52:00, 2021-11-28 15:54:00}|ETH-BTC   |0.07514214285714285|
|{2021-11-28 15:52:00, 2021-11-28 15:54:00}|ETH-USD   |4087.999074074075  |
|{2021-11-28 15:53:00, 2021-11-28 15:55:00}|ETH-USD   |4087.991234

21/11/28 15:54:13 ERROR Executor: Exception in task 12.0 in stage 375.0 (TID 29697)
java.lang.IllegalStateException: Error committing version 71 into HDFSStateStore[id=(op=0,part=12),dir=file:/tmp/temporary-1e6a7ac3-d2bb-4c81-977a-618686ec7c2a/state/0/12]
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider$HDFSBackedStateStore.commit(HDFSBackedStateStoreProvider.scala:148)
	at org.apache.spark.sql.execution.streaming.state.StreamingAggregationStateManagerBaseImpl.commit(StreamingAggregationStateManager.scala:89)
	at org.apache.spark.sql.execution.streaming.StateStoreSaveExec.$anonfun$doExecute$7(statefulOperators.scala:394)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.timeTakenMs(Utils.scala:605)
	at org.apache.spark.sql.execution.streaming.StateStoreWriter.timeTakenMs(statefulOperators.scala:142)
	at org.apache.spark.sql.execution.streaming.StateStoreWriter.timeTakenMs$(statefulOperators.scala:1

# Zadanie 2

**Watermarking i dane opóźnione (3p).** 
Zmodyfikuj zapytanie z zadania 1 tak, aby zademonstować mechanizm znaków wodnych (watermarks) i obsługi danych opóźnionych. W konsoli powinno być widać, że aktualizują się odpowiednie wiersze tabeli wynikowej (tryb update), w szczególności aktualizacja wcześniejszych okien czasowych po przybyciu danych opóźnionych. **Do rozwiązania tego zadania proszę dołączyć przykładowy output i jego opis wyjaśniający na konkretnym przykładzie działanie znaku wodnego i danych opóźnionych**. 

Do ćwiczenia można wykorzystać skrypt w katalogu `/mock` napisany w [Scala-cli](https://scala-cli.virtuslab.org), który posłuży jako kontrolowane źródło danych CoinBase przez Websocket. 

Skrypt można uruchomić wykorzystując Docker:

```
make image
make run
```

Spowoduje to utworzenie websocketowego serwera pod adresem `ws://mock:8025`

Po uruchomieniu serwera należy wykonać poniższą komórkę, w której zapytanie czyta dane z utworzonego websocketa. Skrypt wysyła przykładowe wiadomości w formacie CoinBase co 10 sekund:

- W pierwszej serii wysyłane wiadomości o znacznikach czasowych 0s, 14s, 7s  
- W drugiej serii wysyłane są wiadomości o znacznikach czasowych 15s, 8s, 21s  
- W trzeciej serii wysyłane są wiadomości o znacznikach czasowych 4s, 17s  

Dla tych danych można ustawić okno czasowe na interwał 10 sekund. Skrypt można też zmodyfikować, tak aby wysyłał inne dane. 


In [18]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.sql.streaming.schemaInference", True).getOrCreate()

# 
#     withWatermark("time", "10 minutes").\
# Note url option!
stream = spark.readStream.\
    format("ws").\
    option("schema", "ticker").\
    option("url", "ws://mock:8025").load() # we pass explicit url option to subscribe to our mock service

# your code here...

query = stream.select("product_id", "price", "time").\
    withWatermark("time", "30 seconds").\
    groupBy(window("time", "10 seconds", "5 seconds"), "product_id").mean("price")

query = query.\
    writeStream.\
    format("console").\
    outputMode("update").\
    option("truncate", "false").\
    start()

query.awaitTermination(45) # Let's wait for 10 seconds.
query.stop() # Let's stop the query


21/11/28 16:50:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-96fa0720-c44f-469c-a353-924f29b3a66b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/11/28 16:50:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
21/11/28 16:50:11 WARN WSMicroBatchStreamer: Websocket failed: Failed to connect to mock/172.18.0.3:8025
21/11/28 16:50:11 WARN WSMicroBatchStreamer: Attempting to reconnect in 1s...
21/11/28 16:50:12 WARN WSMicroBatchStreamer: Websocket failed: Failed to connect to mock/172.18.0.3:8025
21/11/28 16:50:12 WARN WSMicroBatchStreamer: Attempting to reconnect in 1s...
21/11/28 16:50:13 WARN WSMicroBatchStreamer: Websocket failed: Failed to connect to mock/172.18.0.3:8025


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg(price)        |
+------------------------------------------+----------+------------------+
|{2021-11-01 00:00:05, 2021-11-01 00:00:15}|ETH-USD   |399.4730625654985 |
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |152.98990174004473|
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |792.2649023736866 |
|{2021-10-31 23:59:55, 2021-11-01 00:00:05}|ETH-USD   |299.29858072277915|
+------------------------------------------+----------+------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg(price)        |
+------------------------------------------+----------+------------------+
|{2021-11-01 00:00:05, 2021-11-01 00:00:15}|ETH-USD   |409.6771738172817 |
|{2021-11-01 00:00:20, 2021-11-01 00:00:30}|ETH-USD   |447.9999289221571 |
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |245.35506660031254|
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |803.5703845996779 |
|{2021-11-01 00:00:15, 2021-11-01 00:00:25}|ETH-USD   |631.4378978739132 |
+------------------------------------------+----------+------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+



-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+----------+-----------------+
|window                                    |product_id|avg(price)       |
+------------------------------------------+----------+-----------------+
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |391.5362987332185|
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |825.5791297770799|
|{2021-10-31 23:59:55, 2021-11-01 00:00:05}|ETH-USD   |564.6892879273577|
|{2021-11-01 00:00:15, 2021-11-01 00:00:25}|ETH-USD   |710.8241386265701|
+------------------------------------------+----------+-----------------+



## Faza 1
Dane są 3 updaty: w 1)00, 2)07, 3)14, które pojawiają się w 4 oknach czasowych:
- 23:59:55 - 00:00:00  -> 1)
- 00:00:00 - 00:00:10  -> 1), 2)
- 00:00:05 - 00:00:15  -> 2), 3)
- 00:00:10 - 00:00:20  -> 3)


## Faza 2
Dane są 3 updaty: w 1)08, 2)15, 3)21, które pojawiają się w 4 oknach czasowych:
- 23:59:55 - 00:00:00  -> --         brak updatu
- 00:00:00 - 00:00:10  -> 1)         update
- 00:00:05 - 00:00:15  -> 1)         update
- 00:00:10 - 00:00:20  -> 2)         update
- 00:00:15 - 00:00:25  -> 2), 3)     update - nowe okno
- 00:00:20 - 00:00:30  -> 3)         update - nowe okno



## Faza 3
Dane są 2 updaty: w 1)04, 2)17, które pojawiają się w 4 oknach czasowych:
- 23:59:55 - 00:00:00  -> 1)         update
- 00:00:00 - 00:00:10  -> 1)         update
- 00:00:05 - 00:00:15  -> --         brak updatu
- 00:00:10 - 00:00:20  -> 2)         update
- 00:00:15 - 00:00:25  -> 2)         update
- 00:00:20 - 00:00:30  -> --         brak updatu


Wszystkie aktualizacje są objęte - watermark ma 30 sekund, które obejmuje cały przebieg i wszystkie updates (23:59:55 00:00:25)

# Zadanie 3

**Łączenie strumieni (3p)**. Rozdziel sztucznie dane CoinBase z kanału `ticker` na dwa strumienie (wykorzystując filtrowanie subskrypcji): jeden strumień dla `side="sell"`, drugi dla `side="buy"`. Następnie stwórz zapytanie, które łączy te strumienie i wypisuje transakcje dla danego `product_id`, które występowały po sobie w ciągu 1s.

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

spark = SparkSession.\
    builder.\
    config("spark.sql.streaming.schemaInference", True).\
    config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True).\
    getOrCreate()  

stream = spark.\
    readStream.\
    format("ws").\
    option("schema", "ticker").\
    load()

# your code...
sell = stream.select("side", "product_id", "price", "time").filter(stream.side == "sell")\
    .withColumnRenamed("side", "sell_side")\
    .withColumnRenamed("product_id", "sell_product_id")\
    .withColumnRenamed("price", "sell_price")\
    .withColumnRenamed("time", "sell_time")


buy = stream.select("side", "product_id", "price", "time").filter(stream.side == "buy")\
    .withColumnRenamed("side", "buy_side")\
    .withColumnRenamed("product_id", "buy_product_id")\
    .withColumnRenamed("price", "buy_price")\
    .withColumnRenamed("time", "buy_time")


joined = sell.join(
    buy,
    expr("""
        buy_product_id = sell_product_id AND
        buy_time >= sell_time AND
        buy_time <= sell_time + interval 1 seconds
        """)
)

query = joined.\
    writeStream.\
    format("console").\
    outputMode("append").\
    option("truncate", "false").\
    start()

query.awaitTermination(120) # Let's wait for 10 seconds.
query.stop() # Let's stop the query


21/11/28 17:28:25 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7b9c0886-bd38-41b2-aff4-08b774ea5724. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/11/28 17:28:25 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+---------------+----------+---------+--------+--------------+---------+--------+
|sell_side|sell_product_id|sell_price|sell_time|buy_side|buy_product_id|buy_price|buy_time|
+---------+---------------+----------+---------+--------+--------------+---------+--------+
+---------+---------------+----------+---------+--------+--------------+---------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4072.13   |2021-11-28 17:28:28.883|buy     |ETH-USD       |4071.38  |2021-11-28 17:28:29.312|
|sell     |ETH-USD        |4072.13   |2021-11-28 17:28:28.888|buy     |ETH-USD       |4071.38  |2021-11-28 17:28:29.312|
|sell     |ETH-USD        |4072.13   |2021-11-28 17:28:28.883|buy     |ETH-USD       |4071.33  |2021-11-28 17:28:29.485|
|sell     |ETH-USD        |4072.13   |2021-11-28 17:28:28.888|buy     |ETH-USD       |4071.33  |2021-11-28 17:28:29.485|
|sell     |ETH-USD        |4072.74   |2021-11-28 17:28:2

-------------------------------------------
Batch: 2
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4070.56   |2021-11-28 17:28:32.947|buy     |ETH-USD       |4070.57  |2021-11-28 17:28:33.016|
|sell     |ETH-USD        |4070.56   |2021-11-28 17:28:32.947|buy     |ETH-USD       |4070.57  |2021-11-28 17:28:33.016|
|sell     |ETH-USD        |4070.56   |2021-11-28 17:28:34.259|buy     |ETH-USD       |4069.93  |2021-11-28 17:28:34.418|
|sell     |ETH-USD        |4069.92   |2021-11-28 17:28:34.356|buy     |ETH-USD       |4069.93  |2021-11-28 17:28:34.418|
|sell     |ETH-USD        |4070.56   |2021-11-28 17:28:3

-------------------------------------------
Batch: 3
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4066.97   |2021-11-28 17:28:34.984|buy     |ETH-USD       |4067.17  |2021-11-28 17:28:35.814|
|sell     |ETH-USD        |4066.77   |2021-11-28 17:28:34.984|buy     |ETH-USD       |4067.17  |2021-11-28 17:28:35.814|
|sell     |ETH-USD        |4066.79   |2021-11-28 17:28:35.183|buy     |ETH-USD       |4067.17  |2021-11-28 17:28:35.814|
|sell     |ETH-USD        |4067.17   |2021-11-28 17:28:35.801|buy     |ETH-USD       |4067.17  |2021-11-28 17:28:35.814|
|sell     |ETH-USD        |4070.56   |2021-11-28 17:28:3

...

-------------------------------------------
Batch: 17
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4067.49   |2021-11-28 17:29:56.755|buy     |ETH-USD       |4066.18  |2021-11-28 17:29:57.366|
|sell     |ETH-USD        |4067.33   |2021-11-28 17:29:56.777|buy     |ETH-USD       |4066.18  |2021-11-28 17:29:57.366|
|sell     |ETH-USD        |4067.42   |2021-11-28 17:29:56.801|buy     |ETH-USD       |4066.18  |2021-11-28 17:29:57.366|
|sell     |ETH-USD        |4068.8    |2021-11-28 17:29:56.575|buy     |ETH-USD       |4066.18  |2021-11-28 17:29:57.366|
|sell     |ETH-USD        |4066.87   |2021-11-28 17:29:

-------------------------------------------
Batch: 18
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4064.02   |2021-11-28 17:29:58.853|buy     |ETH-USD       |4063.53  |2021-11-28 17:29:59.834|
|sell     |ETH-USD        |4061.06   |2021-11-28 17:30:03.354|buy     |ETH-USD       |4061.14  |2021-11-28 17:30:04.168|
|sell     |ETH-USD        |4061.23   |2021-11-28 17:30:04.196|buy     |ETH-USD       |4062.4   |2021-11-28 17:30:04.408|
|sell     |ETH-USD        |4061.41   |2021-11-28 17:30:04.277|buy     |ETH-USD       |4062.4   |2021-11-28 17:30:04.408|
|sell     |ETH-USD        |4064.02   |2021-11-28 17:29:

-------------------------------------------
Batch: 19
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4064.24   |2021-11-28 17:30:07.104|buy     |ETH-USD       |4064.88  |2021-11-28 17:30:07.623|
|sell     |ETH-USD        |4064.25   |2021-11-28 17:30:07.155|buy     |ETH-USD       |4064.88  |2021-11-28 17:30:07.623|
|sell     |ETH-USD        |4063.8    |2021-11-28 17:30:08.419|buy     |ETH-USD       |4063.37  |2021-11-28 17:30:09.405|
|sell     |ETH-USD        |4064.24   |2021-11-28 17:30:07.104|buy     |ETH-USD       |4064.85  |2021-11-28 17:30:07.748|
|sell     |ETH-USD        |4064.25   |2021-11-28 17:30:

-------------------------------------------
Batch: 20
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4062.55   |2021-11-28 17:30:14.13 |buy     |ETH-USD       |4063.86  |2021-11-28 17:30:14.762|
|sell     |ETH-USD        |4062.92   |2021-11-28 17:30:11.215|buy     |ETH-USD       |4063.12  |2021-11-28 17:30:12.187|
|sell     |ETH-USD        |4062.55   |2021-11-28 17:30:14.13 |buy     |ETH-USD       |4063.86  |2021-11-28 17:30:14.995|
|sell     |ETH-USD        |4062.55   |2021-11-28 17:30:14.13 |buy     |ETH-USD       |4063.39  |2021-11-28 17:30:14.602|
+---------+---------------+----------+-----------------

21/11/28 17:30:25 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@51eecd19 is aborting.
21/11/28 17:30:25 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@51eecd19 aborted.
21/11/28 17:30:25 WARN Shell: Interrupted while joining on: Thread[Thread-774370,5,main]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.util.Shell.joinThread(Shell.java:1043)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1003)
	at org.apache.hadoop.util.Shell.run(Shell.java:901)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1213)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1307)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1289)
	at org.apache.hadoop.f